In [8]:
from astropy.table import Table, join
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import pandas as pd
import statistics as st


ForGroup = Table.read('../../GAMA_DATA/G3CFoFGroupv10.fits')
print(f"ForGroup : {len(ForGroup)}")
## Clean Data

GroupGal = Table.read('../../GAMA_DATA/G3CGalv10.fits')
# Clean data
GroupGal = GroupGal[GroupGal['GroupID'] > 0 ]
print(f"GroupGal : {len(GroupGal)}")

StellarMasses = Table.read('../../GAMA_DATA/StellarMassesv19.fits')
# Clean Data
#StellarMasses = StellarMasses[StellarMasses['uminusr'] > 0.001]
#StellarMasses = StellarMasses[StellarMasses['logmstar'] > 0.001]
#StellarMasses = StellarMasses[StellarMasses['metal'] > 0.001]


StellarMasses = StellarMasses[StellarMasses['uminusr'] > 0.01]
StellarMasses = StellarMasses[StellarMasses['logmstar'] > 0.01]
StellarMasses = StellarMasses[StellarMasses['metal'] > 0.01]
print(f"StellarMasses : {len(StellarMasses)}")


envMeasures = Table.read('../../GAMA_DATA/EnvironmentMeasuresv05.fits')
print(f"EnvMeasures : {len(envMeasures)}")
# Clean Data
envMeasures = envMeasures[envMeasures['SurfaceDensity'] < 50]
envMeasures = envMeasures[envMeasures['AGEDenParFlag'] == 0]
envMeasures = envMeasures[envMeasures['CountInCylFlag'] == 0]
envMeasures = envMeasures[envMeasures['CountInCyl'] > 0]

envClass = Table.read('../../GAMA_DATA/GalaxiesClassifiedv01.fits')
# Clean Data
visualMorph = Table.read('../../GAMA_Data/VisualMorphologyv03.fits')
print(f"visualMorph : {len(visualMorph)}")
## Clean Data
visualMorphElliptical = visualMorph[visualMorph['ELLIPTICAL_CODE'] == 1]
visualMorphSpiral = visualMorph[visualMorph['ELLIPTICAL_CODE'] == 10]

DataLocalGroup = join(GroupGal,envMeasures,keys='CATAID',join_type='inner')
print(f"Join GroupGal & EnvMeasures {len(DataLocalGroup)}")
print('GroupGal & EnvMeasures')
print(DataLocalGroup.colnames)
print(len(DataLocalGroup))

df = DataLocalGroup.to_pandas()
dfLog10 = df
for m in ['CountInCyl','DistanceTo5nn','SurfaceDensity','AGEDenPar'] :
    dfLog10[m] = np.log10(df[m])
print(dfLog10.columns)    

dfGrpLog10 = dfLog10.groupby('GroupID')['CountInCyl','DistanceTo5nn','SurfaceDensity','AGEDenPar'].sum()    
    
dfc = df.groupby(['GroupID']).size().reset_index(name='Counts')

dfG = ForGroup.to_pandas()
#print(dfG.columns)
dfm = pd.merge(dfGrpLog10, dfc, on=["GroupID", "GroupID"])
df  = pd.merge(dfm,dfG,on=["GroupID", "GroupID"])
print(df.columns)

from sklearn import linear_model
#import statsmodels.api as sm

Y = df['TotRmag']

for m in ['CountInCyl','DistanceTo5nn','SurfaceDensity','AGEDenPar'] :
    X = df[[m,'Counts']] 
    # with sklearn
    regr = linear_model.LinearRegression()
    r = regr.fit(X, Y)
    print(f'{m} Coefficients: {regr.coef_} Intercept: {regr.intercept_}')
    print(dir(regr))
   


    

ForGroup : 26194
GroupGal : 83093
StellarMasses : 114685
EnvMeasures : 67996
visualMorph : 38795
Join GroupGal & EnvMeasures 24217
GroupGal & EnvMeasures
['CATAID', 'RA_1', 'Dec', 'Z', 'Rpetro', 'DM_100_25_75', 'SURVEY_CODE', 'SigErr', 'GroupID', 'RankIterCen', 'SepIterCen', 'CoSepIterCen', 'AngSepIterCen', 'RankBCG', 'SepBCG', 'CoSepBCG', 'AngSepBCG', 'RankCen', 'SepCen', 'CoSepCen', 'AngSepCen', 'RA_2', 'DEC', 'Z_TONRY', 'nQ', 'M_r', 'DistanceToEdge', 'Completeness', 'DistanceTo5nn', 'SurfaceDensity', 'SurfaceDensityErr', 'SurfaceDensityFlag', 'CountInCyl', 'CountInCylErr', 'CountInCylFlag', 'AGEDenPar', 'AGEErr', 'AGEScale', 'AGEDenParFlag']
24217
Index(['CATAID', 'RA_1', 'Dec', 'Z', 'Rpetro', 'DM_100_25_75', 'SURVEY_CODE',
       'SigErr', 'GroupID', 'RankIterCen', 'SepIterCen', 'CoSepIterCen',
       'AngSepIterCen', 'RankBCG', 'SepBCG', 'CoSepBCG', 'AngSepBCG',
       'RankCen', 'SepCen', 'CoSepCen', 'AngSepCen', 'RA_2', 'DEC', 'Z_TONRY',
       'nQ', 'M_r', 'DistanceToEdge', 'Co

/var/folders/cj/z259fmwd41dgzl8mq8nppwd40000gn/T/ipykernel_67763/3028982459.py:60: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dfGrpLog10 = dfLog10.groupby('GroupID')['CountInCyl','DistanceTo5nn','SurfaceDensity','AGEDenPar'].sum()


CountInCyl Coefficients: [ 0.21013747 -0.35539108] Intercept: -20.16567484458984
['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_feature_names', '_check_n_features', '_decision_function', '_estimator_type', '_get_param_names', '_get_tags', '_more_tags', '_preprocess_data', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_residues', '_set_intercept', '_validate_data', 'coef_', 'copy_X', 'feature_names_in_', 'fit', 'fit_intercept', 'get_params', 'intercept_', 'n_features_in_', 'n_jobs', 'normalize', 'positive', 'predict', 'rank_', 'score', 'set_params', 'singular_']
DistanceTo5nn Coefficients: [-0.11540176 -0.132